<a href="https://colab.research.google.com/github/totminaekaterina/Coursework_multitasking_learning/blob/main/MTM_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Теперь код содержит функции для обучения и оценки модели, а также вычисления метрик качества для каждой задачи. Обратите внимание, что для задач классификации используются метрики точность, полнота и F-мера, а для задачи регрессии (STS) используется метрика MSE.

In [ ]:
!pip install --quiet transformers
!pip install --quiet nlp==0.2.0
!pip install --quiet datasets
!pip install datasets

from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import transformers
import nlp
# from torch.utils.data import Dataset
from datasets import load_dataset, Dataset
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.CRITICAL)

import json
from tqdm.notebook import tqdm

tqdm.pandas()

import pandas as pd
from datasets import Dataset

def load_csv_dataset(train_path, val_path, test_path, column_names, encoding="utf-8-sig"):
    train_data = pd.read_csv(train_path, encoding=encoding, sep=';')
    val_data = pd.read_csv(val_path, encoding=encoding, sep=';')
    test_data = pd.read_csv(test_path, encoding=encoding, sep=';')

    train_data.columns = column_names
    val_data.columns = column_names
    test_data.columns = column_names

    train_dataset = Dataset.from_pandas(train_data)
    val_dataset = Dataset.from_pandas(val_data)
    test_dataset = Dataset.from_pandas(test_data)

    return {"train": train_dataset, "validation": val_dataset, "test": test_dataset}

ner_columns = ["id", "text", "ner"]
sentiment_columns = ["id", "text", "sentiment"]
sts_columns = ["sentence1", "sentence2", "similarity_score"]
nli_columns = ["id", "premise", "hypothesis", "label"]
paraphrase_columns = ["sentence1", "sentence2"]
qg_columns = ["id", "text", "generated_question"]
summarization_columns = ["id", "generated_text", "summary"]
title_gen_columns = ["id", "text", "generated_text"]


path_to_ner_train = "Train_Only_Sentence_NER_full_train.csv"
path_to_ner_validation = "Train_Only_Sentence_NER_full_dev.csv"
path_to_ner_test = "Train_Only_Sentence_NER_full_test.csv"

path_to_sentiment_train = "Train_Only_Sentence_SA_train.csv"
path_to_sentiment_validation = "Train_Only_Sentence_SA_test.csv"
path_to_sentiment_test = "Train_Only_Sentence_SA_test.csv"

path_to_sts_train = "Train_Only_Sentence_STS_train.csv"
path_to_sts_validation = "Train_Only_Sentence_STS_dev.csv"
path_to_sts_test = "Train_Only_Sentence_STS_test.csv"

path_to_nli_train = "Train_Only_Sentence_NLI_train.csv"
path_to_nli_validation = "Train_Only_Sentence_NLI_dev.csv"
path_to_nli_test = "Train_Only_Sentence_NLI_test.csv"

path_to_paraphrase_train = "Train_Only_Sentence_Para_train.csv"
path_to_paraphrase_validation = "Train_Only_Sentence_Para_dev.csv"
path_to_paraphrase_test = "Train_Only_Sentence_Para_test.csv"

path_to_qg_train = "Train_Only_Sentence_QG_train.csv"
path_to_qg_validation = "Train_Only_Sentence_QG_test.csv"
path_to_qg_test = "Train_Only_Sentence_QG_test.csv"

path_to_summarization_train = "Train_Only_Sentence_TextSum_train.csv"
path_to_summarization_validation = "Train_Only_Sentence_TextSum_dev.csv"
path_to_summarization_test = "Train_Only_Sentence_TextSum_test.csv"

path_to_title_gen_train = "Train_Only_Sentence_Title_train.csv"
path_to_title_gen_validation = "Train_Only_Sentence_Title_dev.csv"
path_to_title_gen_test = "Train_Only_Sentence_Title_test.csv"


dataset_dict = {
    "ner": load_csv_dataset(
        path_to_ner_train,
        path_to_ner_validation,
        path_to_ner_test,
        ner_columns
    ),
    "sentiment": load_csv_dataset(
        path_to_sentiment_train,
        path_to_sentiment_validation,
        path_to_sentiment_test,
        sentiment_columns
    ),
    "sts": load_csv_dataset(
        path_to_sts_train,
        path_to_sts_validation,
        path_to_sts_test,
        sts_columns
    ),
    "nli": load_csv_dataset(
        path_to_nli_train,
        path_to_nli_validation,
        path_to_nli_test,
        nli_columns
    ),
    "paraphrase": load_csv_dataset(
        path_to_paraphrase_train,
        path_to_paraphrase_validation,
        path_to_paraphrase_test,
        paraphrase_columns
    ),
    "qg": load_csv_dataset(
        path_to_qg_train,
        path_to_qg_validation,
        path_to_qg_test,
        qg_columns
    ),
    "summarization": load_csv_dataset(
        path_to_summarization_train,
        path_to_summarization_validation,
        path_to_summarization_test,
        summarization_columns
    ),
    "title_gen": load_csv_dataset(
        path_to_title_gen_train,
        path_to_title_gen_validation,
        path_to_title_gen_test,
        title_gen_columns
    ),
}


num_ner_labels = 9
num_sentiment_labels = 3
num_sts_labels = 1
num_nli_labels = 3
num_paraphrase_labels = 1
num_qg_labels = 1
num_text_summarization_labels = 1
num_title_generation_labels = 1

from transformers import BertModel, BertConfig
import torch.nn as nn

class MultiTaskModel(nn.Module):
    def __init__(self, config):
        super(MultiTaskModel, self).__init__()
        
        # Общая часть модели для всех задач
        self.bert = BertModel.from_pretrained('MLM_model', config=config)
        
        
        # Выходные слои для каждой задачи
        self.ner_output = nn.Linear(config.hidden_size, num_ner_labels)
        self.sentiment_output = nn.Linear(config.hidden_size, num_sentiment_labels)
        self.sts_output = nn.Linear(config.hidden_size, 1, num_sts_label)
        self.nli_output = nn.Linear(config.hidden_size, num_nli_labels)
        self.paraphrase_output = nn.Linear(config.hidden_size, num_paraphrase_labels)
        self.qg_output = nn.Linear(config.hidden_size, num_qg_labels=1)
        self.summarization_output = nn.Linear(config.hidden_size, num_summarization_labels)
        self.title_gen_output = nn.Linear(config.hidden_size, num_title_gen_labels)

    def forward(self, task_name, input_ids, attention_mask, token_type_ids=None):
        bert_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = bert_output.pooler_output
        
        if task_name == "ner":
            output = self.ner_output(pooled_output)
        elif task_name == "sentiment":
            output = self.sentiment_output(pooled_output)
        elif task_name == "sts":
            output = self.sts_output(pooled_output)
        elif task_name == "nli":
            output = self.nli_output(pooled_output)
        elif task_name == "paraphrase":
            output = self.paraphrase_output(pooled_output)
        elif task_name == "qg":
            output = self.qg_output(pooled_output)
        elif task_name == "summarization":
            output = self.summarization_output(pooled_output)
        elif task_name == "title_gen":
            output = self.title_gen_output(pooled_output)
        else:
            raise ValueError("Invalid task name")
        
        return output


# Загрузка конфигурации
config = BertConfig.from_pretrained('MLM_model')
config.output_hidden_states = True
config.output_attentions = True

# Создание экземпляра модели
multi_task_model = MultiTaskModel(config, num_ner_labels, num_sentiment_labels, num_sts_labels, num_nli_labels, num_paraphrase_labels, num_qg_labels, num_text_summarization_labels, num_title_generation_labels)


num_ner_labels = 9
num_sentiment_labels = 3
num_sts_labels = 1
num_nli_labels = 3
num_paraphrase_labels = 1
num_qg_labels = 1
num_text_summarization_labels = 1
num_title_generation_labels = 1


def get_ner_labels(text, entities, tokenizer, max_length):
    text, ner_tags = text.split(";")
    tokens = tokenizer.tokenize(text)
    labels = ['O'] * len(tokens)
    ner_tags = ner_tags.strip().split()

    for idx, tag in enumerate(ner_tags):
        if idx < len(labels):
            labels[idx] = tag

    labels = tokenizer.encode(labels, add_special_tokens=False, padding='max_length', max_length=max_length, is_split_into_words=True)
    return labels


def encode_examples(examples, tokenizer, max_length, task):
    encoded_inputs = []

    for example in examples:
        if task == 'ner':
            text = example['text']
            entities = example['ner'].split(';')[1].strip().split()
            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            labels = get_ner_labels(encoded_input['input_ids'], entities)
            encoded_input.update({'labels': labels})

        elif task == 'sentiment':
            text = example['text']
            sentiment = example['sentiment']

            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': sentiment})

        elif task == 'sts':
            text1 = example['sentence1']
            text2 = example['sentence2']
            score = example['similarity_score']

            encoded_input = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': score})
            
        elif task == 'nli':
            text1 = example['premise']
            text2 = example['hypothesis']
            score = example['label']

            encoded_input = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': score})

        elif task == 'paraphrase':
            text = example['sentence1']
            paraphrase = example['sentence2']

            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': paraphrase})
            
        elif task == 'qg':
            text = example['text']
            question = example['generated_question']

            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': question})

        elif task == 'summarization':
            text = example['generated_text']
            summary = example['summary']

            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': summary})
            
        elif task == 'title_gen':
            text = example['text']
            generated_text = example['generated_text']

            encoded_input = tokenizer(text, truncation=True, padding='max_length', max_length=max_length)
            encoded_input.update({'labels': generated_text})

        else:
            pass

        encoded_inputs.append(encoded_input)

    return encoded_inputs


from torch.utils.data import Dataset

class TaskDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_length, task):
        self.data = data
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.task = task

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data[idx]
        encoded_input = self.encoded_inputs[idx]
        item = {key: torch.tensor(val) for key, val in encoded_input.items()}
        item['id'] = data['id']
        return item

        encoding = self.tokenizer(text, truncation=True, max_length=self.max_seq_length, padding="max_length")
        
        if self.task == "ner":
            label_encoding = self.tokenizer(label, truncation=True, max_length=self.max_seq_length, padding="max_length", is_split_into_words=True, return_offsets_mapping=True)
            label_tensor = torch.tensor(label_encoding["input_ids"], dtype=torch.long)
        else:
            # Здесь обрабатываются метки для других задач
            label_tensor = torch.tensor(label, dtype=torch.long)

        # Создание словаря с ключами input_ids, attention_mask и labels
        return {
            "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
            "labels": label_tensor,
        }

# Функция для получения DataLoader для задачи
def get_train_dataloader(task, batch_size=1, max_seq_length=128):
    train_data = dataset_dict[task]['train']
    encoded_examples = encode_examples(train_data, tokenizer, max_seq_length, task)
    dataset = TaskDataset(train_data, encoded_examples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    return dataloader

def get_val_dataloader(task, batch_size=1, max_seq_length=128):
    val_data = dataset_dict[task]['validation']
    encoded_examples = encode_examples(val_data, tokenizer, max_seq_length, task)
    dataset = TaskDataset(val_data, encoded_examples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return dataloader

def get_test_dataloader(task, batch_size=1, max_seq_length=128):
    test_data = dataset_dict[task]['test']
    encoded_examples = encode_examples(test_data, tokenizer, max_seq_length, task)
    dataset = TaskDataset(test_data, encoded_examples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return dataloader

from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup

class TaskSpecificModel(MultiTaskModel):
    def __init__(self, config, num_labels):
        super().__init__(config, num_labels, num_labels, num_labels, num_labels, num_labels, num_labels, num_labels, num_labels)

# Создание словаря меток для каждой из задач
task_num_labels = {
    "ner": 9,
    "sentiment": 3,
    "sts": 1,
    "nli": 3,
    "paraphrase": 1,
    "qg": 1,
    "summarization": 1,
    "title_gen": 1,
}

# Создание конфигураций модели для каждой задачи
task_configs = {
    "ner": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "sentiment": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "sts": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "nli": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "paraphrase": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "qg": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "summarization": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
    "title_gen": BertConfig.from_pretrained('MLM_model', num_hidden_layers=8, num_attention_heads=8),
}

# Создание экземпляров модели для каждой задачи с задаче-специфическими гиперпараметрами
task_models = {task: TaskSpecificModel(config, task_num_labels[task]) for task, config in task_configs.items()}

# Настройка параметров оптимизатора и планировщика для каждой задачи
task_specific_learning_rates = {
    "ner": 3e-5,
    "sentiment": 3e-5,
    "sts": 3e-5,
    "nli": 3e-5,
    "paraphrase": 3e-5,
    "qg": 3e-5,
    "summarization": 3e-5,
    "title_gen": 3e-5,
}

task_optimizers = {}
task_schedulers = {}


from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained('MLM_tokenizer')

batch_size = 1
num_epochs = 3
max_seq_length = 128

train_dataloaders = {task: get_train_dataloader(task, batch_size, max_seq_length) for task in task_models.keys()}

for task, model in task_models.items():
    task_optimizers[task] = AdamW(model.parameters(), lr=task_specific_learning_rates[task])
    num_training_steps = len(train_dataloaders[task]) * num_epochs
    num_warmup_steps = int(0.1 * num_training_steps)
    task_schedulers[task] = get_linear_schedule_with_warmup(
        task_optimizers[task],
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
    )

from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_squared_error

# Определение функции потерь для каждой задачи
task_criterion = {
    "ner": nn.CrossEntropyLoss(),
    "sentiment": nn.CrossEntropyLoss(),
    "sts": nn.MSELoss(),
    "nli": nn.CrossEntropyLoss(),
    "paraphrase": nn.CrossEntropyLoss(),
    "qg": nn.CrossEntropyLoss(),
    "summarization": nn.CrossEntropyLoss(),
    "title_gen": nn.CrossEntropyLoss(),
}


# Создание DataLoader для каждой задачи
batch_size = 1
data_collator = DataCollatorWithPadding(tokenizer)
train_dataloader = {task: DataLoader(dataset_dict[task]['train'], batch_size=batch_size, collate_fn=data_collator) for task in task_models}
val_dataloader = {task: DataLoader(dataset_dict[task]['validation'], batch_size=batch_size, collate_fn=data_collator) for task in task_models}


# Функция для обучения модели
def train(model, dataloader, optimizer, scheduler, criterion, device):
    model.train()
    total_loss = 0.0
    
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Функция для оценки модели
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch['input_ids'].to(device), batch['attention_mask'].to(device), batch['labels'].to(device)
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            predictions = torch.argmax(outputs, dim=-1)
            all_predictions.extend(predictions.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss, all_predictions, all_labels

# Обучение и оценка модели для каждой задачи
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

num_epochs = 3


for task in task_models:
    print(f"Training and evaluating model for task: {task}")
    model = task_models[task].to(device)
    optimizer = task_optimizers[task]
    scheduler = task_schedulers[task]
    criterion = task_criterion[task]

    for epoch in range(num_epochs):
        train_loss = train(model, train_dataloader[task], optimizer, scheduler, criterion, device)
    val_loss, val_predictions, val_labels = evaluate(model, val_dataloader[task], criterion, device)

    print(f"Epoch: {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Вычисление метрик качества для каждой задачи
if task in ["ner", "sentiment", "nli", "paraphrase", "qg", "summarization", "title_gen"]:
    accuracy = accuracy_score(val_labels, val_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_predictions, average='weighted')
    print(f"Task: {task}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
elif task == "sts":
    mse = mean_squared_error(val_labels, val_predictions)
    print(f"Task: {task}, MSE: {mse:.4f}")
else:
    print(f"Invalid task: {task}")

print("Training and evaluation completed.")